In [32]:
import os
import gc
import sys
import time

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
import xgboost as xgb

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import pandas_profiling

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import LabelEncoder

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

import xgboost as xgb

from datetime import datetime
import lightgbm as lgb
#import multiprocessing
#import gc
from sklearn.preprocessing import MinMaxScaler  
##smote 데이터 스케일링
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE ##SMOTE 오예~ 

In [34]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [35]:
def read_and_reduce(filename):
    df = pd.read_csv(filename)
    return reduce_mem_usage(df)

In [36]:
def check_and_drop_column(df, column):
    if column in df.columns:
        df.drop(columns=[column], axis=1, inplace=True)

# Preprocessing

In [37]:
trans_train = pd.read_csv('../../../Data/original/train_transaction.csv', index_col='TransactionID')
trans_test = pd.read_csv('../../../Data/original/test_transaction.csv', index_col='TransactionID')
id_train = pd.read_csv('../../../Data/original/train_identity.csv', index_col='TransactionID')
id_test = pd.read_csv('../../../Data/original/test_identity.csv', index_col='TransactionID')

In [38]:
train = trans_train.merge(id_train, how='left', left_index=True, right_index=True)
test = trans_test.merge(id_test, how='left', left_index=True, right_index=True)

## 민태 파트

In [39]:
V_to_keep = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V9', 'V12', 'V13', 'V17', 'V18', 'V19', 'V20', 'V23', 'V24', 'V25', 'V26', 'V29', 'V30', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V42', 'V43', 'V44', 'V45', 'V47', 'V48', 'V49', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V66', 'V67', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V85', 'V86', 'V87', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V105', 'V109', 'V115', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137', 'V139', 'V140', 'V143', 'V145', 'V146', 'V147', 'V149', 'V150', 'V151', 'V152', 'V156', 'V158', 'V159', 'V160', 'V161', 'V162', 'V164', 'V165', 'V166', 'V167', 'V168', 'V169', 'V170', 'V171', 'V172', 'V173', 'V174', 'V175', 'V176', 'V177', 'V178', 'V179', 'V180', 'V182', 'V187', 'V188', 'V189', 'V192', 'V198', 'V199', 'V200', 'V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V218', 'V219', 'V220', 'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V230', 'V231', 'V232', 'V233', 'V234', 'V236', 'V239', 'V243', 'V244', 'V245', 'V246', 'V248', 'V250', 'V251', 'V256', 'V257', 'V258', 'V259', 'V260', 'V261', 'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276', 'V277', 'V278', 'V279', 'V280', 'V281', 'V282', 'V283', 'V284', 'V285', 'V286', 'V287', 'V288', 'V289', 'V290', 'V291', 'V292', 'V293', 'V294', 'V295', 'V296', 'V297', 'V298', 'V299', 'V300', 'V301', 'V303', 'V304', 'V306', 'V307', 'V308', 'V309', 'V310', 'V311', 'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320', 'V321', 'V322', 'V323', 'V324', 'V326', 'V329', 'V331', 'V332', 'V333', 'V335', 'V336', 'V338', 'V339']

In [40]:
for col in train.columns:
    if str(col)[0] == 'V' in col and col not in V_to_keep:
        check_and_drop_column(train, col)
        
for col in test.columns:
    if str(col)[0] == 'V' in col and col not in V_to_keep:
        check_and_drop_column(test, col)

## 진우 파트

In [41]:
for column in ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2']:
    train[column +  '_count_full'] = \
        train[column].map(pd.concat([train[column], test[column]], ignore_index=True).value_counts(dropna=False))
    test[column +  '_count_full'] = \
        test[column].map(pd.concat([train[column], test[column]], ignore_index=True).value_counts(dropna=False))


In [42]:
train['Transaction_day_of_week'] = np.floor((train['TransactionDT'] / (3600 * 24) - 1) % 7)
test['Transaction_day_of_week'] = np.floor((test['TransactionDT'] / (3600 * 24) - 1) % 7)

train['Transaction_hour_of_day'] = np.floor(train['TransactionDT'] / 3600) % 24
test['Transaction_hour_of_day'] = np.floor(test['TransactionDT'] / 3600) % 24

In [43]:
train['TransactionAmt_to_mean_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_std_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('std')
train['TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')

test['TransactionAmt_to_mean_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_std_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('std')
test['TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')

In [44]:
train['emaildomain'] = train[['R_emaildomain', 'P_emaildomain']].apply(lambda x: x[0] if x[0] == np.nan else x[1], axis=1)
test['emaildomain'] = test[['R_emaildomain', 'P_emaildomain']].apply(lambda x: x[0] if x[0] == np.nan else x[1], axis=1)

In [45]:
train = train.drop(columns=['R_emaildomain', 'P_emaildomain'])
test = test.drop(columns=['R_emaildomain', 'P_emaildomain'])

## 준형 파트

In [46]:
# 준형이형꺼 처리
#id_30

def os(df):
    if df == "-999":
        df = -999
    elif "Windows" in df:
        df = "Windows"
    elif "Mac" in df:
        df = "Mac"
    elif "iOS" in df:
        df = "ios"
    elif "Android" in df:
        df = "Android"
    else:
        df = "other"
        
    return df

train["id_30"].fillna("-999", inplace=True)
train["OS"] = train["id_30"].apply(os)

test["id_30"].fillna("-999", inplace=True)
test["OS"] = test["id_30"].apply(os)

#id_31 
# latest 여부 / 브라우저 col 생성
latest_browser_list = ["samsung browser 7.0",
                  "opera 53.0",
                  "mobile safari 10.0",
                  "google search application 49.0",
                  "firefox 60.0",
                  "edge 17.0",
                  "chrome 69.0", 
                  "chrome 67.0 for android",
                  "chrome 63.0 for android",
                  "chrome 63.0 for ios",
                  "chrome 64.0",
                  "chrome 64.0 for android",
                  "chrome 64.0 for ios",
                  "chrome 65.0",
                  "chrome 65.0 for android",
                  "chrome 65.0 for ios",
                  "chrome 66.0",
                  "chrome 66.0 for android",
                  "chrome 66.0 for ios"
                 ]

def latest_browser(df):
    if df == "-999":
        df = -999
    elif df in latest_browser_list:
        df = 1
    else:
        df = 0
    return df

def browser(df):
    if df == "-999":
        df = -999
    elif "chrome" in df:
        df = "chrome"
    elif "safari" in df:
        df = "safari"
    elif "firefox" in df:
        df = "firefox"
    elif "ie" in df:
        df = "ie"
    elif "samsung" in df:
        df = "samsumg"
    elif "Samsung" in df:
        df = "samsumg"
    return df 
    

train["id_31"].fillna("-999", inplace=True)
train["Latest_browser"] = train["id_31"].apply(latest_browser)
train["Browser"] = train["id_31"].apply(browser)

test["id_31"].fillna("-999", inplace=True)
test["Latest_browser"] = test["id_31"].apply(latest_browser)
test["Browser"] = test["id_31"].apply(browser)

In [51]:
y_train = train['isFraud'].copy()
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()

In [52]:
# Fill numeric columns' NaN values with mean
for c in X_train.columns:
    if X_train[c].dtype=='float16' or  X_train[c].dtype=='float32' or  X_train[c].dtype=='float64':
        X_train[c].fillna(X_train[c].mean())
        X_test[c].fillna(X_train[c].mean())

In [53]:

# Fill categorical columns' NaN values with placeholder (-999)
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

In [50]:
X_train[X_train==np.inf] = np.nan
X_train.fillna(X_train.mean(), inplace=True)
X_test[X_test==np.inf] = np.nan
X_test.fillna(X_test.mean(), inplace=True)

KeyboardInterrupt: 

In [54]:
# 모델설정
sm = SMOTE(ratio='auto', kind='regular')

# train데이터를 넣어 복제함
X_resampled, y_resampled = sm.fit_sample(X_train,list(y_train))

print('After OverSampling, the shape of train_X: {}'.format(X_resampled.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(X_resampled.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_resampled==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_resampled==0)))

ValueError: could not convert string to float: 'W'

# 끗

In [ ]:
train.to_csv('./train.csv', index=False)
test.to_csv('./test.csv', index=False)